In [1]:
import os
import numpy as np

In [2]:
datadir = "../data"

In [3]:
msa_file = os.path.join(datadir, "DHFR.aln")

In [4]:
# Read all the lines in the file into a 2D array of type S1
with open(msa_file) as fh:
    arr = np.array([[x for x in line.strip()] for line in fh], np.dtype("S1"))

print("shape =", arr.shape, ",dtype= ", arr.dtype)

shape = (56165, 186) ,dtype=  |S1


In [5]:
# M is the number of sequences
# L is the length
M, L = arr.shape

In [6]:
# the first sequence
arr[0, :].tostring()

b'VRPLNCIVAVSQNMGIGKNGDLPWPPLRNEFKYFQRMTTTSSVEGKQNLVIMGRKTWFSIPEKNRPLKDRINIVLSRELKEPPRGAHFLAKSLDDALRLIEQPELASKVDMVWIVGGSSVYQEAMNQPGHLRLFVTRIMQEFESDTFFPEIDLGKYKLLPEYPGVLSEVQEEKGIKYKFEVYEKKD'

In [7]:
# the second sequence
arr[1, :].tostring()

b'----SIVVVMCKRFGIGRNGVLPWSPLQADMQRFRSITAG-------GGVIMGRTTFDSIPEEHRPLQGRLNVVLTTSADLMKNSNIIFVSSFDELDAIVGL----HDHLPWHVIGGVSVYQHFLEKSQVTSMYVTFVDGSLECDTFFPHQFLSHFEITRA---SALMSDTTSGMSYRFVDYTR--'

In [8]:
progress_bar = True
try:
    from IPython.display import clear_output
except ImportError:
     progress_bar = False


In [9]:
hamming_cutoff = 0.2 # This is x in equation 27 in the 2013 Coco et al. paper

weights_file = os.path.join(datadir, "DHFR.weights.npy")

if os.path.isfile(weights_file):
    weights = np.load(weights_file)
    print("Loading weights from : ", weights_file)

else:
    weights = np.zeros(M, dtype='f')

    for i in range(M):
        weights[i] = 1. / (np.sum(np.sum(arr[i, :] != arr, axis=1) < hamming_cutoff * L))
        if i % 100 == 0:
            if progress_bar:
                clear_output(wait=True)
            print ("Processing sequence", i, "of", M)
    np.save(weights_file, weights)
    print("Finished computing sequence weights and saved to : ", weights_file)


Loading weights from :  ../data/DHFR.weights.npy


In [12]:
M_eff = sum(weights) # Equation 28 in 2013 Coco et al. paper
print(int(round(M_eff)))

15238


In [14]:
q = 21
pseudo_count = round(M_eff)